<center> <h1> PROJECT : "SPARK with SCALA" </h1> </center>

<center> <h2> Recommendation System on Movies </h2> </center>

<center> <h3> 2019/12/26 </h3> </center>

<center> <h3> Kaelig Castor </h3> </center>

<p style='text-align: justify;'> <b> 
This part of the project is coded in scala. <br> 
<br>
The recommendation system relies on a classical dataset grouping blockbuster ratings. The dataset is already cleaned and labelled. <br> 
<br>
Please change the following data path to access the files (movies.csv and ratings.csv) and run the notebook.</p> </b> 

In [1]:
val path_to_data : String = "/mnt/d/04_EXPLORATION_GRANDS_VOLUMES_DONNEES/ml-latest-small/"

Intitializing Scala interpreter ...

Spark Web UI available at http://172.27.96.1:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1577571979192)
SparkSession available as 'spark'


path_to_data: String = /mnt/d/04_EXPLORATION_GRANDS_VOLUMES_DONNEES/ml-latest-small/


#### First, we define the configuration and initialize the SPARK session.

In [2]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.types.IntegerType
val conf = new SparkConf().setAll(Map(
      "spark.scheduler.mode" -> "FIFO",
      "spark.speculation" -> "false",
      "spark.reducer.maxSizeInFlight" -> "48m",
      "spark.serializer" -> "org.apache.spark.serializer.KryoSerializer",
      "spark.kryoserializer.buffer.max" -> "1g",
      "spark.shuffle.file.buffer" -> "32k",
      "spark.default.parallelism" -> "12",
      "spark.sql.shuffle.partitions" -> "12"
    ))
val spark = SparkSession
      .builder
      .config(conf)
      .appName("Projet Spark")
      .getOrCreate()

import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.types.IntegerType
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@58e37bd4
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@77086eff


#### Second, we read the data files, create the dataframes, and check everything.

In [3]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import org.apache.spark.sql.SQLImplicits
val dfR: DataFrame = spark
      .read
      .option("header", true)         
      .option("inferSchema", "true")  
      .csv(path_to_data + "ratings.csv")
println(s"Nombre de lignes : ${dfR.count}")
println(s"Nombre de colonnes : ${dfR.columns.length}")
dfR.show() // Display the ratings  
dfR.printSchema()
val dfRatings: DataFrame = dfR
      .withColumn("userId" , $"userId".cast("Int"))
      .withColumn("movieId", $"movieId".cast("Int"))
      .withColumn("rating", $"rating".cast("Int"))
      .withColumn("timestamp", $"timestamp".cast("Int"))
dfRatings.printSchema()

val dfM: DataFrame = spark
      .read
      .option("header", true)         
      .option("inferSchema", "true")  
      .csv(path_to_data + "movies.csv")
println(s"Nombre de lignes : ${dfM.count}")
println(s"Nombre de colonnes : ${dfM.columns.length}")
dfM.show() // Display the movies  
dfM.printSchema()
val dfMovies: DataFrame = dfM
      .withColumn("movieId", $"movieId".cast("Int"))
dfMovies.printSchema()

Nombre de lignes : 100836
Nombre de colonnes : 4
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (null

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@1caf54ee
import org.apache.spark.sql.SQLImplicits
dfR: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]
dfRatings: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]
dfM: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]
dfMovies: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]


#### Collaborative Filtering (CF)

<p style='text-align: justify;'> In the following we use one of the most classical collaborative filtering technique. Reminding that collaborative filtering is commonly used for recommender systems, <b>Collaborative Filtering (CF)</b> is a method of making <b>automatic predictions</b> about the interests of a user by learning its preferences (or taste) based on information of his engagements with a set of available items, along with other users’ engagements with the same set of items. In other words, CF assumes that, if a person A has the <b>same opinion as</b> person B on some set of issues X={x1,x2,…}, then A is more likely to have B's opinion on a new issue y than to have the opinion of any other person that doesn’t agree with A on X.</p> 

<p style='text-align: justify;'> So for example, while person A's favorite movies are {<i>American Hustle</i>, <i>Hunger Games</i> and <i>Delivery Man</i>}, person B really likes {<i>American Hustle</i>, <i>Hunger Games</i> and <i>Captain Philips</i>} and person C just loves {<i>12 Years a Slave</i>, <i>Reasonable Doubt</i> and <i>Die Hard II</i>}, following CF approach it should be safer to assume person B should also like <i>Delivery Man</i> and person A would have liked to watch <i>Captain Philips</i>, than to assume any of them would be interested in watching <i>12 Years a Slave</i>, <i>Reasonable Doubt</i> or <i>Die Hard II</i>.</p> 

<p style='text-align: justify;'> By discovering usage/engagement patterns among users and items in the data, <b>CF algorithms are able to infer users’ hidden preferences and to exploit those preferences to recommend them new potentially-good items.</b> CF algorithms are best known for their use on e-commerce web sites, where they serve as cornerstones for their recommendation-engines.</p> 

<p style='text-align: justify;'> First, we describe one of the most popular CF algorithm, known as <b>Matrix Factorization.</b> Then we will describe the <b>Alternating Least Squares minimization.</b></p> 


#### Matrix Factorization

<p style='text-align: justify;'> One of the most popular algorithms to solve co-clustering problems (and specifically for collaborative recommender systems) is called Matrix Factorization (MF). In its simplest form, it assumes a matrix ${{R}\in{R^{m \times n}}}$ of ratings given by musers to nitems. Applying this technique on $R$ will end up factorizing $R$ into two matrices ${{U}\in{R^{m \times k}}}$ and ${{P}\in{R^{n \times k}}}$ such that $R \approx U \times P$ (their multiplication approximates $R$).</p> 


#### Alternating Least Squares minimization

<p style='text-align: justify;'> Alternating minimization represents a widely applicable and empirically successful approach for finding low-rank matrices that best fit the given data. Looking again at MF’s cost function, it appears that we aim at learning two types of variables – those of $U$ and those of $P$, and the two types are tied in the multiplication of $U \times {P^T}$. Recall that the actual cost function is the $\sum ||R-{U}\times{P^T}||_2$ = $\mathop \sum\limits_{i,j}\left({R_{i,j}-{u_i}\times{p_j}}\right)$ plus regularization term. The fact that both $U$’s and $V$’s values are unknown variables makes this cost function non-convex.</p> 

<p style='text-align: justify;'> But another interesting fact lies in this term – if we fix $P$ and optimize for $U$ alone, the problem is simply reduced to the problem of linear regression. Recall that in linear regression, we solve for $\theta$ by minimizing the squared error $||y - X\theta||_2$ given $X$ and $y$. The solution is ultimately given by the Ordinary Least Squares (OLS) estimator ${\hat{\theta} = \left({{X^T}X}\right)^{ - 1}}{X^T}y$.</p> 

<p style='text-align: justify;'> Alternating least squares does just that. It is <b>a two-step iterative optimization process.</b> In every iteration, it first fixes $P$ and solves for $U$, and following that it fixes $U$ and solves for $P$. Since OLS solution is unique and guarantees a minimal MSE, in each step the cost function can either decrease or stay unchanged, but never increase. Alternating between the two steps guarantees reduction of the cost function, until convergence. Similarly to gradient descent optimization, it is guaranteed to converge only to a local minima, and is ultimately depends on initial values for $U$ or $P$.</p> 

<p style='text-align: justify;'> Since the actual cost function includes a regularization term, it is slightly longer. According to the two-step process, the cost function can be broken down into two cost functions:</p> 

$\forall{u_i}: J\left({u_i}\right) = ||R_i - {u_i}\times{P^T}||_2 + \lambda \cdot ||u_i||_2$

$\forall{p_j}: J\left({p_j}\right) = ||R_i - U\times{p_j^T}||_2 + \lambda \cdot ||p_j||_2$

And the matching solutions for ${u_i}$ and ${p_j}$ are:

${u_i} = {\left( {{P^T} \times P + \lambda I} \right)^{ - 1}} \times {P^T} \times {R_i}$

${p_j} = {\left( {{U^T} \times U + \lambda I} \right)^{ - 1}} \times {U^T} \times {R_j}$

<p style='text-align: justify;'> And since each ${u_i}$ doesn’t depend on other ${u_{j \ne i}}$ vectors, each step can potentially be introduced to massive parallelization.</p> 

In [4]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

case class Rating(userId: Int, movieId: Int, rating: Float, timestamp: Long)
def parseRating(str: String): Rating = {
              val fields = str.split("::")
              assert(fields.size == 4)
              Rating(fields(0).toInt, fields(1).toInt, fields(2).toFloat, fields(3).toLong)
}

// SPLITTING THE DATASET INTO TRAINING (80%) and TEST (20%) SUBSETS
val Array(training, test) = dfRatings.randomSplit(Array(0.8, 0.2))

// Build the recommendation model using ALS (Alternating Least Squares) matrix factorization on the training data
val als = new ALS()
  .setMaxIter(5)
  .setRegParam(0.01)
  .setImplicitPrefs(true)
  .setUserCol("userId")
  .setItemCol("movieId")
  .setRatingCol("rating")

// Training the ALS model
val model = als.fit(training) 

// Evaluate the model by computing the RMSE on the test data
// Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
model.setColdStartStrategy("drop")
val predictions = model.transform(test)  

val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating")
  .setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

// Generate top 10 movie recommendations for each user
val userRecs = model.recommendForAllUsers(10)
userRecs.show()

Root-mean-square error = 3.1013445961764288
+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    50|[[4993, 0.9183656...|
|   280|[[4306, 0.8846774...|
|   300|[[318, 0.6642463]...|
|   400|[[318, 0.8859384]...|
|   161|[[595, 0.37635446...|
|   411|[[590, 1.0840641]...|
|   471|[[318, 0.6441616]...|
|   591|[[2571, 0.5389139...|
|   172|[[593, 0.27602068...|
|   232|[[33679, 1.301014...|
|   282|[[2571, 1.2251546...|
|   452|[[1240, 1.1373057...|
|   482|[[780, 0.71582305...|
|    73|[[68954, 1.060413...|
|   233|[[318, 1.1297597]...|
|   263|[[4306, 1.0056376...|
|   343|[[318, 0.7205763]...|
|   363|[[318, 0.5212913]...|
|   403|[[2571, 0.2521280...|
|   413|[[318, 0.69170713...|
+------+--------------------+
only showing top 20 rows



import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
defined class Rating
parseRating: (str: String)Rating
training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
als: org.apache.spark.ml.recommendation.ALS = als_44d84b341671
model: org.apache.spark.ml.recommendation.ALSModel = als_44d84b341671
predictions: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_f825b2ea5167
rmse: Double = 3.1013445961764288
userRecs: org.apache.spark.sql.DataFrame = [userId: int, recommendations:...

In [5]:
// Generate top 10 user recommendations for each movie
val movieRecs = model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     14|[[474, 0.89270496...|
|     18|[[599, 1.0080756]...|
|     25|[[387, 1.2729722]...|
|     38|[[448, 0.5665329]...|
|     46|[[474, 0.94490904...|
|     50|[[28, 1.2722144],...|
|     73|[[599, 0.83765906...|
|     97|[[599, 0.81554574...|
|    161|[[6, 1.3349816], ...|
|    172|[[599, 1.4691492]...|
|    186|[[6, 0.98019737],...|
|    225|[[6, 1.275411], [...|
|    232|[[606, 0.8815048]...|
|    233|[[599, 0.86460394...|
|    248|[[599, 0.73354214...|
|    254|[[6, 0.17176375],...|
|    257|[[414, 1.0522056]...|
|    263|[[606, 0.52587324...|
|    280|[[474, 0.63333803...|
|    282|[[6, 1.14901], [1...|
+-------+--------------------+
only showing top 20 rows



movieRecs: org.apache.spark.sql.DataFrame = [movieId: int, recommendations: array<struct<userId:int,rating:float>>]


In [6]:
// Generate top 10 movie recommendations for a specified set of users
val users = dfRatings.select(als.getUserCol).distinct().limit(10)
val userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    50|[[4993, 0.9183656...|
|   161|[[595, 0.37635446...|
|   172|[[593, 0.27602068...|
|    73|[[68954, 1.060413...|
|    14|[[296, 0.7601203]...|
|    25|[[318, 0.5504094]...|
|    46|[[296, 0.8334836]...|
|    97|[[2571, 0.5067856...|
|    18|[[318, 1.4194739]...|
|    38|[[356, 0.96146435...|
+------+--------------------+



users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int]
userSubsetRecs: org.apache.spark.sql.DataFrame = [userId: int, recommendations: array<struct<movieId:int,rating:float>>]


In [7]:
// Generate top 10 user recommendations for a specified set of movies
val movies = dfRatings.select(als.getItemCol).distinct().limit(10)
val movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     50|[[28, 1.2722144],...|
|   1030|[[474, 1.0218881]...|
|   1092|[[414, 1.0558394]...|
|   2093|[[414, 0.7096334]...|
|   1214|[[469, 1.3113378]...|
|   2414|[[414, 0.8183437]...|
|   1256|[[448, 0.92395514...|
|   2366|[[448, 0.9029837]...|
|    919|[[474, 1.4919729]...|
|   1029|[[448, 0.9125159]...|
+-------+--------------------+



movies: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [movieId: int]
movieSubSetRecs: org.apache.spark.sql.DataFrame = [movieId: int, recommendations: array<struct<userId:int,rating:float>>]
